# Creación de conversaciones ICMP

### Descripción: 
Con este código se va a intentar crear todos los pasos de una conversación ARP, mediante IA generativa. Se pretende pasar una descripción de esta conversación al modelo y que este sea capaz de crear todos los pasos/paquetes necesarios para replicar una conversación con esas mismas especificaciones

In [ ]:
import random
import re
import pickle
import time

import openai
openai.api_key = "FILL"

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.inet import IP, ICMP

openai.__version__

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


'1.10.0'

## Generación de la descipción de una comunicación ARP

<center><img src="https://lh3.googleusercontent.com/proxy/5BFPjEuLzirf_rQUpK9oPibFDhfXjP7eez9zJdPXfEl5QZyEp6jLTDSpVc8RBygmwlQ5s4hV-V_muXLdOGmmrsjD-JaqO4HtlQHkO1D8Tj8">
    
<center> Esta imagen es genérica, nosotros crearemos conversaciones que sean tanto una pregunta a broadcast como a una ubicación específica.
<br>
    ---
<br>Una conversación normal ARP tiene una petición (opcode = 1) en la que una fuente pregunta a broadcast o a otro elemento de la red por una IP en concreto, y diciendo la IP a la que comunicar la respuesta.
La respuesta tiene (opcode = 2), y tiene como fuente las respuestas, mientras que como destino tiene al que inicia la conversación.   
<center>---

La primera query (query1) representa una conexión ARP sencilla donde se envía la petición a un equipo específico, la segunda (query2) es más complicada, enviando la solicitud a broadcast.

In [2]:
query = "This is an ICMP Echo conversation summary:\n"
query += 'Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 seq= 81 ttl=56 type=Echo"\n'
query += "This is the python code to generate the request and reply packets of the conversation, respectively:\n"
query += 'pkt_1=scapy.IP(src="192.168.1.10", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=8, id=0x0045, seq=81)\n'
query += 'time.sleep(abs(random.gauss(0, 0.03)))'
query += 'pkt_2=scapy.IP(src="192.168.1.20", dst="192.168.1.10", ttl=56)/scapy.ICMP(type=0, id=0x0045, seq=81)\n'

query += "This is an ICMP Timestamp conversation summary:\n"
query += 'Source: IP="222.240.4.204" // Destination: IP="192.168.1.20"// Others: id=0xFF62 seq= 4311 ttl=56 type=Timestamp"\n'
query += "This is the python code to generate the request and reply packets of the conversation, respectively:\n"
query += 'pkt_1=scapy.IP(src="222.240.4.204", dst="200.40.184.21", ttl=56)/scapy.ICMP(type=13, id=0xFF62, seq=4311)\n'
query += 'time.sleep(abs(random.gauss(0, 0.03)))'
query += 'pkt_2=scapy.IP(src="200.40.184.21", dst="222.240.4.204", ttl=56)/scapy.ICMP(type=14, id=0xFF62, seq=4311)\n'

In [3]:
print(query)

This is an ICMP Echo conversation summary:
Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 seq= 81 ttl=56 type=Echo"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="192.168.1.10", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=8, id=0x0045, seq=81)
time.sleep(abs(random.gauss(0, 0.03)))pkt_2=scapy.IP(src="192.168.1.20", dst="192.168.1.10", ttl=56)/scapy.ICMP(type=0, id=0x0045, seq=81)
This is an ICMP Timestamp conversation summary:
Source: IP="222.240.4.204" // Destination: IP="192.168.1.20"// Others: id=0xFF62 seq= 4311 ttl=56 type=Timestamp"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="222.240.4.204", dst="200.40.184.21", ttl=56)/scapy.ICMP(type=13, id=0xFF62, seq=4311)
time.sleep(abs(random.gauss(0, 0.03)))pkt_2=scapy.IP(src="200.40.184.21", dst="222.240.4.204", ttl=56)/scapy.ICMP(type=14, id=0xFF62, seq=431

## Carga de posibles conversaciones

In [4]:
with open("../../data/Conversations/ICMP/ICMP_Conv_Testing_v2.txt","r",encoding="utf8") as f:
    Conv_summaries = f.read().splitlines()
    
print("Número de resúmenes: " +str(len(Conv_summaries)))

Número de resúmenes: 280


In [5]:
test_group = random.choices(Conv_summaries,k=200)

### Carga de posibles respuestas anteriores para encadenarlas

In [6]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [7]:
try:
    responses = pickle.load( open("../../data/Conversations/ICMP/pickle/ICMP_pairs_Aday_validation_v3.pkl", "rb" ))
except:
    responses = []

print("Number of previous responses: " + str(len(responses)))

Number of previous responses: 0


## Generación de comandos de paquetes de conversación

In [9]:
index = 0

for i in range(len(test_group)):
   
    summary = test_group[i]
    
    query1=query    
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter " +str(i+1) +")")
    

    ## Luego seleccionas aleatoriamente resumenes ya generados para pedirle que te saque el comando a partir del paquete
    #summaries = random.choices(Conv_summary,k=summaries_per_request)
    text_sum = ""
    
    query1 += "This is a conversation summary:\n"
    query1 += summary + "\n"
    text_sum += summary + "\n"
    query1 += "\nGenerate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.\n"
    query1 += "Use the following structure:\n"
    query1 += "pkt_1=...\n"
    query1 += "time.sleep(abs(random.gauss(0, 0.03)))\n"
    query1 += "pkt_2=...\n"
    
    #Query es el mensaje que le envias al modelo
    print(query1)

    #Esta es la petición que le mandas al modelo de IA
    completion = openai.completions.create(
        model="gpt-3.5-turbo-instruct", 
        prompt=query1,
        max_tokens=2600,
        temperature=0.1)


    
    print("...Generated!")
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(text_sum, completion))
    
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open( "../../data/Conversations/ICMP/pickle/ICMP_pairs_Aday_validation_v3.pkl", "wb" ) )
    
    exec(responses[-1].completion.choices[0].text)
    
    with open("../../data/Conversations/ICMP/pcap/ICMP_v3_generated_3_5_turbo-instruct_conv_v3.pcap", "ba+") as f:
        wrpcap(f, pkt_1, append=True)
    
    with open("../../data/Conversations/ICMP/pcap/ICMP_v3_generated_3_5_turbo-instruct_conv_v3.pcap", "ba+") as f:
        wrpcap(f, pkt_2, append=True)
    
    print("Number of completitions done: " + str(len(responses)))

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 1)
This is an ICMP Echo conversation summary:
Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 seq= 81 ttl=56 type=Echo"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="192.168.1.10", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=8, id=0x0045, seq=81)
time.sleep(abs(random.gauss(0, 0.03)))pkt_2=scapy.IP(src="192.168.1.20", dst="192.168.1.10", ttl=56)/scapy.ICMP(type=0, id=0x0045, seq=81)
This is an ICMP Timestamp conversation summary:
Source: IP="222.240.4.204" // Destination: IP="192.168.1.20"// Others: id=0xFF62 seq= 4311 ttl=56 type=Timestamp"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="222.240.4.204", dst="200.40.184.21", ttl=56)/scapy.ICMP(type=13, id=0xFF62, seq=4311)
time.sleep(abs(random.gauss(0

...Generated!
Number of completitions done: 6
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 7)
This is an ICMP Echo conversation summary:
Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 seq= 81 ttl=56 type=Echo"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="192.168.1.10", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=8, id=0x0045, seq=81)
time.sleep(abs(random.gauss(0, 0.03)))pkt_2=scapy.IP(src="192.168.1.20", dst="192.168.1.10", ttl=56)/scapy.ICMP(type=0, id=0x0045, seq=81)
This is an ICMP Timestamp conversation summary:
Source: IP="222.240.4.204" // Destination: IP="192.168.1.20"// Others: id=0xFF62 seq= 4311 ttl=56 type=Timestamp"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="222.240.4.204", dst="200.40.184.21", ttl=56)/scapy.ICMP(type=13, id=0

...Generated!
Number of completitions done: 12
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 13)
This is an ICMP Echo conversation summary:
Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 seq= 81 ttl=56 type=Echo"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="192.168.1.10", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=8, id=0x0045, seq=81)
time.sleep(abs(random.gauss(0, 0.03)))pkt_2=scapy.IP(src="192.168.1.20", dst="192.168.1.10", ttl=56)/scapy.ICMP(type=0, id=0x0045, seq=81)
This is an ICMP Timestamp conversation summary:
Source: IP="222.240.4.204" // Destination: IP="192.168.1.20"// Others: id=0xFF62 seq= 4311 ttl=56 type=Timestamp"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="222.240.4.204", dst="200.40.184.21", ttl=56)/scapy.ICMP(type=13, id

...Generated!
Number of completitions done: 18
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 19)
This is an ICMP Echo conversation summary:
Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 seq= 81 ttl=56 type=Echo"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="192.168.1.10", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=8, id=0x0045, seq=81)
time.sleep(abs(random.gauss(0, 0.03)))pkt_2=scapy.IP(src="192.168.1.20", dst="192.168.1.10", ttl=56)/scapy.ICMP(type=0, id=0x0045, seq=81)
This is an ICMP Timestamp conversation summary:
Source: IP="222.240.4.204" // Destination: IP="192.168.1.20"// Others: id=0xFF62 seq= 4311 ttl=56 type=Timestamp"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="222.240.4.204", dst="200.40.184.21", ttl=56)/scapy.ICMP(type=13, id

...Generated!
Number of completitions done: 24
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 25)
This is an ICMP Echo conversation summary:
Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 seq= 81 ttl=56 type=Echo"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="192.168.1.10", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=8, id=0x0045, seq=81)
time.sleep(abs(random.gauss(0, 0.03)))pkt_2=scapy.IP(src="192.168.1.20", dst="192.168.1.10", ttl=56)/scapy.ICMP(type=0, id=0x0045, seq=81)
This is an ICMP Timestamp conversation summary:
Source: IP="222.240.4.204" // Destination: IP="192.168.1.20"// Others: id=0xFF62 seq= 4311 ttl=56 type=Timestamp"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="222.240.4.204", dst="200.40.184.21", ttl=56)/scapy.ICMP(type=13, id

...

...Generated!
Number of completitions done: 174
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 175)
This is an ICMP Echo conversation summary:
Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 seq= 81 ttl=56 type=Echo"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="192.168.1.10", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=8, id=0x0045, seq=81)
time.sleep(abs(random.gauss(0, 0.03)))pkt_2=scapy.IP(src="192.168.1.20", dst="192.168.1.10", ttl=56)/scapy.ICMP(type=0, id=0x0045, seq=81)
This is an ICMP Timestamp conversation summary:
Source: IP="222.240.4.204" // Destination: IP="192.168.1.20"// Others: id=0xFF62 seq= 4311 ttl=56 type=Timestamp"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="222.240.4.204", dst="200.40.184.21", ttl=56)/scapy.ICMP(type=13, 

...Generated!
Number of completitions done: 180
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 181)
This is an ICMP Echo conversation summary:
Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 seq= 81 ttl=56 type=Echo"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="192.168.1.10", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=8, id=0x0045, seq=81)
time.sleep(abs(random.gauss(0, 0.03)))pkt_2=scapy.IP(src="192.168.1.20", dst="192.168.1.10", ttl=56)/scapy.ICMP(type=0, id=0x0045, seq=81)
This is an ICMP Timestamp conversation summary:
Source: IP="222.240.4.204" // Destination: IP="192.168.1.20"// Others: id=0xFF62 seq= 4311 ttl=56 type=Timestamp"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="222.240.4.204", dst="200.40.184.21", ttl=56)/scapy.ICMP(type=13, 

...Generated!
Number of completitions done: 186
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 187)
This is an ICMP Echo conversation summary:
Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 seq= 81 ttl=56 type=Echo"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="192.168.1.10", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=8, id=0x0045, seq=81)
time.sleep(abs(random.gauss(0, 0.03)))pkt_2=scapy.IP(src="192.168.1.20", dst="192.168.1.10", ttl=56)/scapy.ICMP(type=0, id=0x0045, seq=81)
This is an ICMP Timestamp conversation summary:
Source: IP="222.240.4.204" // Destination: IP="192.168.1.20"// Others: id=0xFF62 seq= 4311 ttl=56 type=Timestamp"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="222.240.4.204", dst="200.40.184.21", ttl=56)/scapy.ICMP(type=13, 

...Generated!
Number of completitions done: 192
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 193)
This is an ICMP Echo conversation summary:
Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 seq= 81 ttl=56 type=Echo"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="192.168.1.10", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=8, id=0x0045, seq=81)
time.sleep(abs(random.gauss(0, 0.03)))pkt_2=scapy.IP(src="192.168.1.20", dst="192.168.1.10", ttl=56)/scapy.ICMP(type=0, id=0x0045, seq=81)
This is an ICMP Timestamp conversation summary:
Source: IP="222.240.4.204" // Destination: IP="192.168.1.20"// Others: id=0xFF62 seq= 4311 ttl=56 type=Timestamp"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="222.240.4.204", dst="200.40.184.21", ttl=56)/scapy.ICMP(type=13, 

...Generated!
Number of completitions done: 198
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 199)
This is an ICMP Echo conversation summary:
Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 seq= 81 ttl=56 type=Echo"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="192.168.1.10", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=8, id=0x0045, seq=81)
time.sleep(abs(random.gauss(0, 0.03)))pkt_2=scapy.IP(src="192.168.1.20", dst="192.168.1.10", ttl=56)/scapy.ICMP(type=0, id=0x0045, seq=81)
This is an ICMP Timestamp conversation summary:
Source: IP="222.240.4.204" // Destination: IP="192.168.1.20"// Others: id=0xFF62 seq= 4311 ttl=56 type=Timestamp"
This is the python code to generate the request and reply packets of the conversation, respectively:
pkt_1=scapy.IP(src="222.240.4.204", dst="200.40.184.21", ttl=56)/scapy.ICMP(type=13, 

In [72]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion.choices[0].text)

Source: IP="192.168.1.1" // Destination: IP="192.168.1.2" // Others: id=0x4952 seq= 1303



pkt_list = []

# Request packet
request_pkt = scapy.IP(src="192.168.1.1", dst="192.168.1.2")/scapy.ICMP(type=8, id=0x4952, seq=1303)
pkt_list.append(request_pkt)

# Reply packet
reply_pkt = scapy.IP(src="192.168.1.2", dst="192.168.1.1")/scapy.ICMP(type=0, id=0x4952, seq=1303)
pkt_list.append(reply_pkt)


In [75]:
#Esta es la forma que tiene una respuesta del modelo de IA
print(responses[-1].completion)

Completion(id='cmpl-9GtCQbzvShHTmYNT96kDftxV8YxqR', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n\npkt_list = []\n\n# Request packet\nrequest_pkt = scapy.IP(src="192.168.1.1", dst="192.168.1.2")/scapy.ICMP(type=8, id=0x4952, seq=1303)\npkt_list.append(request_pkt)\n\n# Reply packet\nreply_pkt = scapy.IP(src="192.168.1.2", dst="192.168.1.1")/scapy.ICMP(type=0, id=0x4952, seq=1303)\npkt_list.append(reply_pkt)')], created=1713812462, model='gpt-3.5-turbo-instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=119, prompt_tokens=230, total_tokens=349))


### Generación de paquetes

In [78]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec(responses[-1].completion.choices[0].text)
print(pkt_list)

[<IP  frag=0 proto=icmp src=192.168.1.1 dst=192.168.1.2 |<ICMP  type=echo-request id=0x4952 seq=0x517 |>>, <IP  frag=0 proto=icmp src=192.168.1.2 dst=192.168.1.1 |<ICMP  type=echo-reply id=0x4952 seq=0x517 |>>]


In [80]:
#Enviarlos para probar si se envian correctamente
send(pkt_list)


Sent 2 packets.


### Introducción en un pcap

In [83]:
# Guardamos los paquetes generados en un pcap
with open("../../data/Conversations/ICMP/pcap/ICMPgenerated_3_5_turbo-instruct_conv.pcap", "wb") as f:
    wrpcap(f, pkt_list)